*참고자료*
- https://nurilee.com/2020/04/03/lightgbm-definition-parameter-tuning/

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb


# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc
sns.set(font="THEGaeideuk",rc={"axes.unicode_minus":False},style='white')

## 데이터 로드

In [2]:
err=pd.read_csv('./data/train_err_data.csv')
quality=pd.read_csv('./data/train_quality_data.csv')
problem=pd.read_csv('./data/train_problem_data.csv')
test_err=pd.read_csv('./data/test_err_data.csv')
test_quality=pd.read_csv('./data/test_quality_data.csv')

C:\Users\oye01\anaconda3\envs\pydatavenv\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\oye01\anaconda3\envs\pydatavenv\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (4,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train=[i for i in range(10000,25000)]
test=[i for i in range(30000,44999)]
train=pd.DataFrame(train,columns=['user_id'])
test=pd.DataFrame(test,columns=['user_id'])

In [4]:
err.isna().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     1
dtype: int64

In [5]:
test_err.isna().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     4
dtype: int64

In [6]:
err

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0
...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,15,1
16554659,24999,20201130172625,model_3,05.15.2138,16,1
16554660,24999,20201130172625,model_3,05.15.2138,4,0
16554661,24999,20201130172631,model_3,05.15.2138,4,0


### model컬럼

In [7]:
# train
model_col=pd.get_dummies(err[['user_id','model_nm']],columns=['model_nm']).groupby(['user_id']).sum()
model_col=(model_col>0).reset_index()
train=pd.merge(train,model_col)

# test
model_col=pd.get_dummies(test_err[['user_id','model_nm']],columns=['model_nm']).groupby(['user_id']).sum()
model_col=(model_col>0).reset_index()
test=pd.merge(test,model_col)

<br>

### fwver

In [8]:
# train
fwver_col=pd.get_dummies(err[['user_id','fwver']],columns=['fwver']).groupby(['user_id']).sum()
fwver_col=(fwver_col>0).reset_index()
train=pd.merge(train,fwver_col)

# test
fwver_col=pd.get_dummies(test_err[['user_id','fwver']],columns=['fwver']).groupby(['user_id']).sum()
fwver_col=(fwver_col>0).reset_index()
test=pd.merge(test,fwver_col)

MemoryError: Unable to allocate 4.56 GiB for an array with shape (16554663, 37) and data type float64

<br>

### errtype

In [ ]:
# train
errtype_col=pd.get_dummies(err[['user_id','errtype']],columns=['errtype']).groupby(['user_id']).sum()
errtype_col=(errtype_col>0).reset_index()
train=pd.merge(train,errtype_col)

# test
errtype_col=pd.get_dummies(test_err[['user_id','errtype']],columns=['errtype']).groupby(['user_id']).sum()
errtype_col=(errtype_col>0).reset_index()
test=pd.merge(test,errtype_col)

In [ ]:
###################
err[(err['user_id']==10000)&(err['time']>=20201129080000)]

<br>

### errcode

In [ ]:
# train
err_val=err['errcode'].value_counts(normalize=True).cumsum()[:20].index
err.loc[~(err['errcode'].isin(err_val).values),'errcode']='etc'

In [ ]:
# test
test_err.loc[~(test_err['errcode'].isin(err_val).values),'errcode']='etc'

In [ ]:
# train
errcode_col=pd.get_dummies(err[['user_id','errcode']],columns=['errcode']).groupby(['user_id']).sum()
errcode_col=(errcode_col>0).reset_index()
train=pd.merge(train,errcode_col)

# test
errcode_col=pd.get_dummies(test_err[['user_id','errcode']],columns=['errcode']).groupby(['user_id']).sum()
errcode_col=(errcode_col>0).reset_index()
test=pd.merge(test,errcode_col)

<br>

### problem

In [ ]:
prob_col=np.array(np.zeros(15000))
prob_col[problem['user_id'].unique()-10000]=1

In [ ]:
prob_col

<br><br>

## Modeling

In [ ]:
X=train.loc[:,'model_nm_model_0':'errcode_standby']
y=prob_col
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.75,random_state=2020)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
X_train

In [ ]:
light_gbm=lgb.Dataset(X_train, label=y_train)

In [ ]:
params={'learning_rate':0.003,'objective':'binary','metric':'binary_logloss','max_depth':15}
clf = lgb.train(params, light_gbm, 3000)

In [ ]:
y_pred=clf.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_pred[y_pred>0.5]=1
y_pred[y_pred<=0.5]=0

In [ ]:
sum(y_pred==1)

In [ ]:
accuracy_score(y_pred,y_test)

<br>

## Submission

In [ ]:
test.loc[:,'model_nm_model_0':'errcode_standby']

In [ ]:
test=test.loc[:,'model_nm_model_0':'errcode_standby']

In [ ]:
light_gbm=lgb.Dataset(X, label=y)

In [ ]:
params={'learning_rate':0.003,'objective':'binary','metric':'binary_logloss','max_depth':15}
clf = lgb.train(params, light_gbm, 3000)

In [268]:
clf.predict(test.loc[:,'model_nm_model_0':'errcode_standby'])

LightGBMError: The number of features in data (111) is not the same as it was in training data (108).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.